In [48]:
import requests
import os
import json
import random 
import time
from datetime import datetime
import psycopg2
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

In [49]:
# Database connection parameters
dbname = "colruyt_tracker"
user = "thibodebras"
password = "postgres"
host = "localhost"  # or the IP address of your PostgreSQL server
port = "5432"

In [94]:
conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=host)

query = "SELECT * FROM raw_data_2024_04_03"

# Use read_sql_query to execute the query and assign the result to a DataFrame
df_raw = pd.read_sql_query(query, conn)

# Don't forget to close the connection when done
conn.close()

/var/folders/9n/plp7r74s4fn_nthngkbm1gzr0000gn/T/ipykernel_40312/633877234.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_raw = pd.read_sql_query(query, conn)


In [95]:
df_raw.head()

,run_date,shop_id,page,products
0,2024-04-03,459,1,"[{'productId': '843058', 'technicalArticleNumb..."
1,2024-04-03,459,2,"[{'productId': '96', 'technicalArticleNumber':..."
2,2024-04-03,459,3,"[{'productId': '2127', 'technicalArticleNumber..."
3,2024-04-03,459,4,"[{'productId': '7250', 'technicalArticleNumber..."
4,2024-04-03,459,5,"[{'productId': '14867', 'technicalArticleNumbe..."


In [96]:
exploded_df = df_raw.explode('products')

products_normalized = pd.json_normalize(exploded_df['products'].apply(lambda x: x if isinstance(x, dict) else {}))

# Drop the 'products' column from exploded_df since it's no longer needed in its original form
exploded_df.drop(columns=['products'], inplace=True)

# Reset the index of exploded_df to merge it with products_normalized
exploded_df.reset_index(drop=True, inplace=True)

# Concatenate the normalized products DataFrame with the exploded DataFrame
# It's important to reset the index on products_normalized if you haven't already, to ensure alignment
result_df = pd.concat([exploded_df, products_normalized.reset_index(drop=True)], axis=1)


In [97]:
result_df[result_df['productId']=='211038']

,run_date,shop_id,page,productId,technicalArticleNumber,commercialArticleNumber,name,brand,seoBrand,thumbNail,fullImage,squareImage,content,walkRouteSequenceNumber,isAvailable,isPriceAvailable,inPromo,gtin,topCategoryName,topCategoryId,businessDomain,IsPrivateLabel,IsBiffe,WeightconversionFactor,IsWeightArticle,IsBio,CountryOfOrigin,IsExclusivelySoldInLuxembourg,OrderUnit,ShortName,IsNew,LongName,RecentQuanityOfStockUnits,AlcoholVolume,FicCode,price.basicPrice,price.recommendedQuantity,price.measurementUnitPrice,price.measurementUnit,price.isRedPrice,price.pricePerUOM,price.activationDate,price.recordSource,price.isPromoActive,price.priceChangeCode,description,nutriscoreLabel,price.unit,price.quantityPrice,price.quantityPriceQuantity,price.measurementUnitQuantityPrice,price.quantityPricePerUOM,price.quantityActivationDate,price.quantityPriceChangeCode,InSeason,StartSeasonDate,price.redPriceReason,EcoscoreLabel,EcoscoreValue,promotion,DepositValue,EndSeasonDate,price.referencePrice,price.retentionPeriod,BiffeDate
3524,2024-04-03,459,15,211038,3524212,33317,boulettes apéro Excell. ±40pc,MORA,Mora,https://static.colruytgroup.com/images/200x200...,https://static.colruytgroup.com/images/500x500...,https://static.colruytgroup.com/images/500x500...,800g,15232.0,True,True,False,"[08710861086052, 08710861986055]",Surgelés,129,RETAIL_BE,False,False,0,False,False,PAYS-BAS,False,P,MORA EX BOUL.APERO40P,False,MORA boulettes apéro Excell. ±40pc 800g,"1,12",0,E,4.99,1.0,6.24,K,False,6.24,27-03-2024,Offline,N,E,*7% de viande de b\nuf\n*Friteuse,D,NaN,3.74,2.0,4.68,4.68,27-03-2024,L,NaN,NaN,NaN,E,0,NaN,NaN,NaN,NaN,NaN,NaN
17335,2024-04-03,1825,15,211038,3524212,33317,boulettes apéro Excell. ±40pc,MORA,Mora,https://static.colruytgroup.com/images/200x200...,https://static.colruytgroup.com/images/500x500...,https://static.colruytgroup.com/images/500x500...,800g,14739.0,True,True,False,"[08710861086052, 08710861986055]",Surgelés,129,RETAIL_BE,False,False,0,False,False,PAYS-BAS,False,P,MORA EX BOUL.APERO40P,False,MORA boulettes apéro Excell. ±40pc 800g,"1,12",0,E,4.99,1.0,6.24,K,False,6.24,27-03-2024,Offline,N,E,*7% de viande de b\nuf\n*Friteuse,D,NaN,3.74,2.0,4.68,4.68,27-03-2024,L,NaN,NaN,NaN,E,0,NaN,NaN,NaN,NaN,NaN,NaN
31146,2024-04-03,471,15,211038,3524212,33317,boulettes apéro Excell. ±40pc,MORA,Mora,https://static.colruytgroup.com/images/200x200...,https://static.colruytgroup.com/images/500x500...,https://static.colruytgroup.com/images/500x500...,800g,NaN,False,False,False,"[08710861086052, 08710861986055]",Surgelés,129,RETAIL_BE,False,False,0,False,False,PAYS-BAS,False,P,MORA EX BOUL.APERO40P,False,MORA boulettes apéro Excell. ±40pc 800g,"1,12",0,E,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,*7% de viande de b\nuf\n*Friteuse,D,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,E,0,NaN,NaN,NaN,NaN,NaN,NaN
44957,2024-04-03,1671,15,211038,3524212,33317,boulettes apéro Excell. ±40pc,MORA,Mora,https://static.colruytgroup.com/images/200x200...,https://static.colruytgroup.com/images/500x500...,https://static.colruytgroup.com/images/500x500...,800g,15634.0,True,True,False,"[08710861086052, 08710861986055]",Surgelés,129,RETAIL_BE,False,False,0,False,False,PAYS-BAS,False,P,MORA EX BOUL.APERO40P,False,MORA boulettes apéro Excell. ±40pc 800g,"1,12",0,E,4.99,1.0,6.24,K,False,6.24,27-03-2024,Offline,N,E,*7% de viande de b\nuf\n*Friteuse,D,NaN,3.74,2.0,4.68,4.68,27-03-2024,L,NaN,NaN,NaN,E,0,NaN,NaN,NaN,NaN,NaN,NaN


In [57]:
# Explode the 'products' column, which turns each element of a list-like element to a row
exploded_df = df_raw.explode('products')

# Normalize the nested dictionaries within the 'products' column
products_normalized = pd.json_normalize(
    exploded_df['products'].apply(lambda x: x if isinstance(x, dict) else {})
)

# Drop the 'products' column as it is no longer needed
exploded_df.drop(columns=['products'], inplace=True)

# Concatenate the DataFrame with normalized product details
# Resetting the index just before concatenation to ensure proper alignment
result_df = pd.concat([exploded_df.reset_index(drop=True), products_normalized], axis=1)

In [56]:
result_df1.equals(result_df2)

True

In [21]:
result_df.head()

,run_date,shop_id,page,productId,technicalArticleNumber,commercialArticleNumber,name,brand,seoBrand,thumbNail,fullImage,squareImage,content,walkRouteSequenceNumber,isAvailable,isPriceAvailable,inPromo,gtin,topCategoryName,topCategoryId,businessDomain,IsPrivateLabel,IsBiffe,WeightconversionFactor,IsWeightArticle,IsBio,CountryOfOrigin,IsExclusivelySoldInLuxembourg,OrderUnit,ShortName,IsNew,LongName,RecentQuanityOfStockUnits,AlcoholVolume,FicCode,price.basicPrice,price.recommendedQuantity,price.measurementUnitPrice,price.measurementUnit,price.isRedPrice,price.pricePerUOM,price.activationDate,price.recordSource,price.isPromoActive,price.priceChangeCode,description,nutriscoreLabel,price.unit,price.quantityPrice,price.quantityPriceQuantity,price.measurementUnitQuantityPrice,price.quantityPricePerUOM,price.quantityActivationDate,price.quantityPriceChangeCode,InSeason,StartSeasonDate,price.redPriceReason,EcoscoreLabel,EcoscoreValue,promotion,DepositValue,EndSeasonDate,price.referencePrice,price.retentionPeriod,BiffeDate
0,2024-04-03,459,1,843058,4688137,30635,élevées au sol M,EVERYDAY,Everyday,https://static.colruytgroup.com/images/200x200...,https://static.colruytgroup.com/images/500x500...,https://static.colruytgroup.com/images/500x500...,12pc,8910.0,True,True,False,"[35400141530941, 45400141530948, 7540014153094...",Crémerie,65,RETAIL_BE,True,False,0,False,False,BELGIQUE,False,P,EVD SCHARRELEI M 12ST,False,EVERYDAY élevées au sol M 12pc,"1,15,15,15,15,15",0,I,2.22,1.0,0.185,S,False,0.185,17-01-2024,Offline,N,L,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2024-04-03,459,1,2204,29013,29013,bananes,BONI,Boni Selection,https://static.colruytgroup.com/images/200x200...,https://static.colruytgroup.com/images/500x500...,https://static.colruytgroup.com/images/500x500...,±1kg,9366.0,True,True,False,"[05400141963946, 05400141978933, 0540014153206...",Légumes et fruits,1675,RETAIL_BE,True,False,1000,True,False,GUATEMALA,False,P,BONI BANANEN W,False,BONI bananes ±1kg,"1,1,1,1,1",0,I,1.99,1.2,1.990,K,False,1.990,03-04-2024,Offline,N,E,"*Origine: Guatemala,\nÉquateur,\nCosta Rica, C...",A,K,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2024-04-03,459,1,955,29522,29522,poivrons rouges,NaN,NaN,https://static.colruytgroup.com/images/200x200...,https://static.colruytgroup.com/images/500x500...,https://static.colruytgroup.com/images/500x500...,NaN,9815.0,True,True,False,"[98436570312680, 05400141295221, 0542501649044...",Légumes et fruits,1675,RETAIL_BE,False,False,250,True,False,ESPAGNE,False,"P,Kg",POIVRON ROUGE W,False,poivrons rouges,"1,1",0,N,2.99,1.0,2.990,K,False,2.990,26-03-2024,Offline,N,H,"*Origine: Espagne\n*Classe 1\n*Wok, salades,\n...",NaN,K,2.870,3.0,2.870,2.870,26-03-2024,H,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2024-04-03,459,1,1038,29750,29750,concombre,NaN,NaN,https://static.colruytgroup.com/images/200x200...,https://static.colruytgroup.com/images/500x500...,https://static.colruytgroup.com/images/500x500...,NaN,9839.0,True,True,False,"[05400141095814, 05400357000916, 0541464329750...",Légumes et fruits,1675,RETAIL_BE,False,False,0,False,False,BELGIQUE,False,P,CONCOMBRES PIECE,False,concombre,"1,12",NaN,E,0.89,1.0,0.890,S,False,0.890,03-04-2024,Offline,N,E,"*Origine: Espagne, Belgique\n*Classe 1",NaN,NaN,0.445,2.0,0.445,0.445,03-04-2024,L,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2024-04-03,459,1,197525,3440812,29018,bananes,PAPILLON,Papillon,https://static.colruytgroup.com/images/200x200...,https://static.colruytgroup.com/images/500x500...,https://static.colruytgroup.com/images/500x500...,±1kg,9365.0,True,True,False,"[95425016493536, 05425016493533, 9540014100913...",Légumes et fruits,1675,RETAIL_BE,False,False,1000,True,False,GUATEMALA,False,P,BANANES PAPILLON P,False,PAPILLON bananes ±1kg,1,0,E,2.40,1.2,2.400,K,False,2.400,02-11-2023,Offline,N,H,"*Origine: Guatemala, Costa Rica\n*Classe 1\n*V...",NaN,K,2.200,3.4,2.200,2.200,02-11-2023,H,True,26/02/2016,NaN,Na

In [58]:
result_df = result_df.drop_duplicates(subset=["shop_id", "productId"])

In [59]:
def create_tables(conn):
    # SQL statements for creating the tables
    commands = (
        """
        CREATE TABLE IF NOT EXISTS Product (
            ProductID INT PRIMARY KEY,
            Name VARCHAR(255),
            LongName VARCHAR(255),
            ShortName VARCHAR(100),
            Description TEXT,
            TopCategoryName VARCHAR(255),
            TopCategoryId INT,
            Brand VARCHAR(255),
            SeoBrand VARCHAR(255)
        );
        """,
        """
        CREATE TABLE IF NOT EXISTS Shop (
            ShopID INT PRIMARY KEY,
            ShopName VARCHAR(255) NOT NULL,
            Address VARCHAR(255),
            City VARCHAR(100)
        );
        """,
        """
        CREATE TABLE IF NOT EXISTS PriceHistory (
            PriceID VARCHAR(32) PRIMARY KEY,
            EffectiveDate DATE NOT NULL,
            ProductID INT NOT NULL,
            ShopID INT NOT NULL,
            BasicPrice DECIMAL(10, 2),
            RecommendedQuantity DECIMAL(10, 2),
            QuantityPrice DECIMAL(10, 2),
            QuantityPriceQuantity DECIMAL(10, 2),
            MeasurementUnitPrice DECIMAL(10, 2),
            MeasurementUnitQuantityPrice DECIMAL(10, 2),
            MeasurementUnit VARCHAR(50),
            IsRedPrice BOOLEAN,
            PricePerUOM DECIMAL(10, 2),
            ActivationDate VARCHAR(50),
            RecordSource VARCHAR(50),
            IsPromoActive VARCHAR(50),
            PriceChangeCode VARCHAR(10),
            QuantityPricePerUOM DECIMAL(10, 2),
            QuantityActivationDate VARCHAR(50),
            QuantityPriceChangeCode VARCHAR(10),
            RedPriceReason VARCHAR(255),
            ReferencePrice DECIMAL(10, 2),
            RetentionPeriod DECIMAL(10, 2),
            Unit VARCHAR(50),
            FOREIGN KEY (ProductID) REFERENCES Product(ProductID),
            FOREIGN KEY (ShopID) REFERENCES Shop(ShopID)
        );
        """,
        """
        CREATE TABLE IF NOT EXISTS ProductHistory (
            ProductHistoryID VARCHAR(32) PRIMARY KEY,
            EffectiveDate DATE NOT NULL,
            ProductID INT NOT NULL,
            IsAvailable BOOLEAN,
            IsPriceAvailable BOOLEAN,
            InPromo BOOLEAN,
            IsBio BOOLEAN,
            CountryOfOrigin VARCHAR(100),
            IsNew BOOLEAN,
            Content TEXT,
            NutriscoreLabel VARCHAR(10),
            EcoscoreLabel VARCHAR(10),
            EcoscoreValue DECIMAL(5, 2),
            FOREIGN KEY (ProductID) REFERENCES Product(ProductID)
        );
        """
    )
    cur = conn.cursor()
    try:
        # Execute each command
        for command in commands:
            cur.execute(command)
        cur.close()
        conn.commit()  # Commit the changes to the database
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        conn.rollback()  # Rollback on error
    finally:
        if cur is not None:
            cur.close()


In [75]:

conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=host)
create_tables(conn)
conn.close()

In [76]:

# Call the function to create the tables
#

In [77]:
product_description_mask = [
    'productId', 
    'name', 
    'LongName', 
    'ShortName', 
    'description', 
    'topCategoryName', 
    'topCategoryId', 
    'brand', 
    'seoBrand'
]
product_variables_mask = [
    'run_date',
    'productId', 
    'isAvailable', 
    'isPriceAvailable', 
    'inPromo', 
    'IsBio', 
    'CountryOfOrigin', 
    'IsNew', 
    'content', 
    'nutriscoreLabel', 
    'EcoscoreLabel', 
    'EcoscoreValue'
]

price_mask = [
    'run_date',
    'productId', 
    'shop_id',
    'price.basicPrice', 
    'price.recommendedQuantity', 
    'price.quantityPrice', 
    'price.quantityPriceQuantity', 
    'price.measurementUnitPrice', 
    'price.measurementUnitQuantityPrice', 
    'price.measurementUnit', 
    'price.isRedPrice', 
    'price.pricePerUOM', 
    'price.activationDate', 
    'price.recordSource', 
    'price.isPromoActive', 
    'price.priceChangeCode', 
    'price.quantityPricePerUOM', 
    'price.quantityActivationDate', 
    'price.quantityPriceChangeCode', 
    'price.redPriceReason', 
    'price.referencePrice', 
    'price.retentionPeriod', 
    'price.unit', 
    # 'promotion'
]



In [78]:

product_description_columns = {
    'productId': 'productId', 
    'name': 'name', 
    'LongName': 'longName', 
    'ShortName': 'shortName', 
    'description': 'description', 
    'topCategoryName': 'topCategoryName', 
    'topCategoryId': 'topCategoryId', 
    'brand': 'brand', 
    'seoBrand': 'seoBrand'
}

product_variables_columns = {
    'run_date': "effectiveDate",
    'productId': 'productId', 
    'isAvailable': 'isAvailable', 
    'isPriceAvailable': 'isPriceAvailable', 
    'inPromo': 'inPromo', 
    'IsBio': 'isBio', 
    'CountryOfOrigin': 'countryOfOrigin', 
    'IsNew': 'isNew', 
    'content': 'content', 
    'nutriscoreLabel': 'nutriscoreLabel', 
    'EcoscoreLabel': 'ecoscoreLabel', 
    'EcoscoreValue': 'ecoscoreValue'
}

price_columns = {
    'price_id':"priceId",
    'run_date': "effectiveDate",
    'productId': 'productId', 
    'shop_id': 'shopId',
    'price.basicPrice': 'basicPrice', 
    'price.recommendedQuantity': 'recommendedQuantity', 
    'price.quantityPrice': 'quantityPrice', 
    'price.quantityPriceQuantity': 'quantityPriceQuantity', 
    'price.measurementUnitPrice': 'measurementUnitPrice', 
    'price.measurementUnitQuantityPrice': 'measurementUnitQuantityPrice', 
    'price.measurementUnit': 'measurementUnit', 
    'price.isRedPrice': 'isRedPrice', 
    'price.pricePerUOM': 'pricePerUOM', 
    'price.activationDate': 'activationDate', 
    'price.recordSource': 'recordSource', 
    'price.isPromoActive': 'isPromoActive', 
    'price.priceChangeCode': 'priceChangeCode', 
    'price.quantityPricePerUOM': 'quantityPricePerUOM', 
    'price.quantityActivationDate': 'quantityActivationDate', 
    'price.quantityPriceChangeCode': 'quantityPriceChangeCode', 
    'price.redPriceReason': 'redPriceReason', 
    'price.referencePrice': 'referencePrice', 
    'price.retentionPeriod': 'retentionPeriod', 
    'price.unit': 'unit'
}



In [79]:
import hashlib
import pandas as pd
import numpy as np

# Process Products DataFrame
products_df = (
    result_df[product_description_mask]
    .drop_duplicates()
    .replace({np.nan: None})
    .rename(columns=product_description_columns)
)
products_df['productId'] = products_df['productId'].astype(int)
products_df.reset_index(drop=True, inplace=True)

# Process Product History DataFrame
products_history_df = (
    result_df[product_variables_mask]
    .drop_duplicates()
    .rename(columns=product_variables_columns)
)
products_history_df.reset_index(drop=True, inplace=True)

# Process Prices DataFrame
prices_df = (
    result_df[price_mask]
    .rename(columns=price_columns)
    .assign(
        isRedPrice=lambda df: df['isRedPrice'].fillna(False),
        shopId=lambda df: df['shopId'].astype(int),
        productId=lambda df: df['productId'].astype(int)
    )
)
prices_df.reset_index(drop=True, inplace=True)

# Process Shops DataFrame
shops_df = pd.DataFrame(result_df['shop_id'].unique(), columns=['shopId'])
shops_df = shops_df.assign(
    shopName=np.nan,
    address=np.nan,
    city=np.nan
)
shops_df.reset_index(drop=True, inplace=True)

# Hashing function for generating IDs
def hash_row(row):
    row_str = ''.join(row.astype(str))
    return hashlib.md5(row_str.encode()).hexdigest()

# Apply the hash function to generate priceId and productHistoryId
exclude_columns = ['effectiveDate']

prices_df['priceId'] = prices_df.drop(columns=exclude_columns).apply(hash_row, axis=1)
products_history_df['productHistoryId'] = products_history_df.drop(columns=exclude_columns).apply(hash_row, axis=1)

/var/folders/9n/plp7r74s4fn_nthngkbm1gzr0000gn/T/ipykernel_40312/1783889404.py:28: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  isRedPrice=lambda df: df['isRedPrice'].fillna(False),


In [80]:
products_history_df[products_history_df['productHistoryId'] == 'a87b7805b84c4dd9fd712bbe1dfff1ea'].head()

,effectiveDate,productId,isAvailable,isPriceAvailable,inPromo,isBio,countryOfOrigin,isNew,content,nutriscoreLabel,ecoscoreLabel,ecoscoreValue,productHistoryId
3474,2024-04-02,211038,True,True,False,False,PAYS-BAS,False,800g,D,E,0,a87b7805b84c4dd9fd712bbe1dfff1ea


In [81]:
products_history_df.head()

,effectiveDate,productId,isAvailable,isPriceAvailable,inPromo,isBio,countryOfOrigin,isNew,content,nutriscoreLabel,ecoscoreLabel,ecoscoreValue,productHistoryId
0,2024-04-02,1038,True,True,False,False,BELGIQUE,False,NaN,NaN,NaN,NaN,ddef41f03a77a1fb5d9134a161b02b49
1,2024-04-02,1027,True,True,False,False,ESPAGNE,False,NaN,NaN,NaN,NaN,81250fd680673144af31929c144f0498
2,2024-04-02,13902,True,True,False,False,BELGIQUE,False,NaN,A,NaN,NaN,ff53c2081fab7f43a34ee39fc0ec6a51
3,2024-04-02,955,True,True,False,False,ESPAGNE,False,NaN,NaN,NaN,NaN,9f2891b840e60560790a93f7d9cbeb7a
4,2024-04-02,1020,True,True,False,False,ESPAGNE,False,500g,A,NaN,NaN,1f600ebbb81f81da9967e450af705fc9


# Update shop and product 

In [82]:
metrics = {
    'total_shop_added': 0,
    'total_product_added': 0,
    'total_price_updates': 0,
    'total_product_updates': 0
}

In [83]:
db = psycopg2.connect(dbname=dbname, user=user, password=password, host=host)

cursor = db.cursor()

# Retrieve current products and shops from the database
cursor.execute("SELECT * FROM Producthistory WHERE producthistoryid = 'a87b7805b84c4dd9fd712bbe1dfff1ea'" )
existing_products = {row[0] for row in cursor.fetchall()}


In [84]:
query = "SELECT * FROM Producthistory WHERE producthistoryid = 'a87b7805b84c4dd9fd712bbe1dfff1ea'"
old = pd.read_sql(query, db)

/var/folders/9n/plp7r74s4fn_nthngkbm1gzr0000gn/T/ipykernel_40312/1903932864.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  old = pd.read_sql(query, db)


In [85]:
old

,producthistoryid,effectivedate,productid,isavailable,ispriceavailable,inpromo,isbio,countryoforigin,isnew,content,nutriscorelabel,ecoscorelabel,ecoscorevalue


In [92]:
query = "SELECT * FROM Producthistory WHERE productid = '211038'"
old = pd.read_sql(query, db)
old

/var/folders/9n/plp7r74s4fn_nthngkbm1gzr0000gn/T/ipykernel_40312/3320091834.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  old = pd.read_sql(query, db)


,id,producthistoryid,effectivedate,productid,isavailable,ispriceavailable,inpromo,isbio,countryoforigin,isnew,content,nutriscorelabel,ecoscorelabel,ecoscorevalue
0,2024-04-02a87b7805b84c4dd9fd712bbe1dfff1ea,a87b7805b84c4dd9fd712bbe1dfff1ea,2024-04-02,211038,True,True,False,False,PAYS-BAS,False,800g,D,E,0.0
1,2024-04-03a68344ddd81da0f404a0be45bec8cc8c,a68344ddd81da0f404a0be45bec8cc8c,2024-04-03,211038,False,False,False,False,PAYS-BAS,False,800g,D,E,0.0
2,2024-04-03a87b7805b84c4dd9fd712bbe1dfff1ea,a87b7805b84c4dd9fd712bbe1dfff1ea,2024-04-03,211038,True,True,False,False,PAYS-BAS,False,800g,D,E,0.0


In [87]:

# Establish database connection
db = psycopg2.connect(dbname=dbname, user=user, password=password, host=host)

cursor = db.cursor()

# Retrieve current products and shops from the database
cursor.execute("SELECT productId FROM Product")
existing_products = {row[0] for row in cursor.fetchall()}

cursor.execute("SELECT shopId FROM Shop")
existing_shops = {row[0] for row in cursor.fetchall()}

# Insert new products if they do not exist
new_products = products_df[~products_df['productId'].astype(int).isin(existing_products)]
for _, row in new_products.iterrows():
    cursor.execute("""
        INSERT INTO Product (productId, name, longName, shortName, description, topCategoryName, topCategoryId, brand, seoBrand)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
    """, (row['productId'], row['name'], row['longName'], row['shortName'], row['description'], row['topCategoryName'], 
          row['topCategoryId'], row['brand'], row['seoBrand']))

# Insert new shops if they do not exist
new_shops = shops_df[~shops_df['shopId'].astype(int).isin(existing_shops)]
for shop_id in new_shops['shopId'].unique():
    # Example shop data, adjust as per actual data
    shop_name = 'Unknown name'
    address = 'Unknown address'
    city = 'Unknown city'
    cursor.execute("""
        INSERT INTO Shop (shopId, shopName, address, city)
        VALUES (%s, %s, %s, %s)
    """, (shop_id, shop_name, address, city))

metrics['total_shop_added'] += len(new_shops)
metrics['total_product_added'] += len(new_products)

db.commit()

cursor.close()
db.close()

In [88]:
metrics

{'total_shop_added': 0,
 'total_product_added': 0,
 'total_price_updates': 0,
 'total_product_updates': 0}

# Update price and product history

In [89]:
# Database connection using a context manager


with psycopg2.connect(dbname=dbname, user=user, password=password, host=host) as db:
    query = """
        SELECT PriceId
        FROM PriceHistory
        WHERE (ProductID, ShopID, EffectiveDate) IN (
            SELECT ProductID, ShopID, MAX(EffectiveDate)
            FROM PriceHistory
            GROUP BY ProductID, ShopID
        )
    """
    existing_price_ids = pd.read_sql(query, db)

    # Filter for price changes
    price_changes = prices_df[~prices_df['priceId'].isin(existing_price_ids['priceid'])]

    # Use a cursor with a context manager
    with db.cursor() as cursor:
        # Batch insert for efficiency if there are many rows
        batch_size = 1000  # Adjust batch size based on your needs
        for i in range(0, len(price_changes), batch_size):
            batch = price_changes.iloc[i:i+batch_size]
            values = [
                (
                    row['effectiveDate'], row['productId'], row['shopId'], row['basicPrice'],
                    row['recommendedQuantity'], row['quantityPrice'], row['quantityPriceQuantity'],
                    row['measurementUnitPrice'], row['measurementUnitQuantityPrice'], row['measurementUnit'],
                    row['isRedPrice'], row['pricePerUOM'], row['activationDate'], row['recordSource'],
                    row['isPromoActive'], row['priceChangeCode'], row['quantityPricePerUOM'],
                    row['quantityActivationDate'], row['quantityPriceChangeCode'], row['redPriceReason'],
                    row['referencePrice'], row['retentionPeriod'], row['unit'], row['priceId']
                    # row['promotion'],
                )
                for _, row in batch.iterrows()
            ]

            # Execute the batch insert
            cursor.executemany("""
                INSERT INTO PriceHistory (EffectiveDate, ProductID, ShopID, BasicPrice, RecommendedQuantity, QuantityPrice,
                                          QuantityPriceQuantity, MeasurementUnitPrice, MeasurementUnitQuantityPrice,
                                          MeasurementUnit, IsRedPrice, PricePerUOM, ActivationDate, RecordSource,
                                          IsPromoActive, PriceChangeCode, QuantityPricePerUOM, QuantityActivationDate,
                                          QuantityPriceChangeCode, RedPriceReason, ReferencePrice, RetentionPeriod,
                                          Unit, PriceId)
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            """, values)
            
            # Update the metrics
            metrics['total_price_updates'] += len(batch)
        
        # Commit the transaction
        db.commit()

/var/folders/9n/plp7r74s4fn_nthngkbm1gzr0000gn/T/ipykernel_40312/3469769800.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  existing_price_ids = pd.read_sql(query, db)


In [90]:
metrics['total_price_updates']

54913

In [91]:
# Database connection using a context manager
with psycopg2.connect(dbname=dbname, user=user, password=password, host=host) as db:
    query = """
        SELECT ProductHistoryId
        FROM ProductHistory
        WHERE (ProductID, EffectiveDate) IN (
            SELECT ProductID, MAX(EffectiveDate)
            FROM ProductHistory
            GROUP BY ProductID
        )
    """
    existing_product_history_ids = pd.read_sql(query, db)

    # Filter for price changes
    product_changes = products_history_df[~products_history_df['productHistoryId'].isin(existing_product_history_ids['producthistoryid'])]


    # Use a cursor with a context manager
    with db.cursor() as cursor:
        # Batch insert for efficiency if there are many rows
        batch_size = 1000  # Adjust batch size based on your needs
        for i in range(0, len(product_changes), batch_size):
            batch = product_changes.iloc[i:i+batch_size]
            values = [
                (
                    row['effectiveDate'], row['productId'], row['isAvailable'], row['isPriceAvailable'],
                    row['inPromo'], row['isBio'], row['countryOfOrigin'], row['isNew'],
                    row['content'], row['nutriscoreLabel'], row['ecoscoreLabel'], row['ecoscoreValue'], row['productHistoryId']
                )
                for _, row in batch.iterrows()
            ]

            # Execute the batch insert
            cursor.executemany("""
                INSERT INTO ProductHistory (EffectiveDate, ProductID, IsAvailable, IsPriceAvailable, InPromo, IsBio,
                                            CountryOfOrigin, IsNew, Content, NutriscoreLabel, EcoscoreLabel, EcoscoreValue, ProductHistoryId)
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            """, values)
            
            # Update the metrics
            metrics['total_product_updates'] += len(batch)

        # Commit the transaction
        db.commit()

/var/folders/9n/plp7r74s4fn_nthngkbm1gzr0000gn/T/ipykernel_40312/4233807432.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  existing_product_history_ids = pd.read_sql(query, db)


In [47]:
product_changes[product_changes['productId'] == '211038']

,effectiveDate,productId,isAvailable,isPriceAvailable,inPromo,isBio,countryOfOrigin,isNew,content,nutriscoreLabel,ecoscoreLabel,ecoscoreValue,productHistoryId
3511,2024-04-03,211038,True,True,False,False,PAYS-BAS,False,800g,D,E,0,a87b7805b84c4dd9fd712bbe1dfff1ea


In [88]:
metrics['total_product_updates']

62

In [89]:
metrics

{'total_shop_added': 0,
 'total_product_added': 14,
 'total_price_updates': 5526,
 'total_product_updates': 62}

In [ ]:
{'total_shop_added': 4,
 'total_product_added': 13810,
 'total_price_updates': 54913,
 'total_product_updates': 16496}